In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('kc_house_data_test_features.csv')
print(df.shape)
df.head()

(4322, 21)


,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [3]:
import pickle

In [4]:
infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [5]:
print(model.intercept_)
print(len(model.coef_))

1550069.1088693272
80


In [6]:
infile = open("other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [7]:
other_info

[Index(['yr_built', 'sqft_living', 'sqft_lot', 'sqft_percentage', 'grade',
        'condition', 'floors', 'bathrooms', 'bedrooms', 'sqft_living15',
        'sqft_lot15', 'zipcode_98058', 'zipcode_98115', 'zipcode_98006',
        'zipcode_98034', 'zipcode_98052', 'zipcode_98031', 'zipcode_98007',
        'zipcode_98199', 'zipcode_98027', 'zipcode_98119', 'zipcode_98122',
        'zipcode_98155', 'zipcode_98055', 'zipcode_98118', 'zipcode_98014',
        'zipcode_98059', 'zipcode_98004', 'zipcode_98053', 'zipcode_98072',
        'zipcode_98133', 'zipcode_98092', 'zipcode_98074', 'zipcode_98125',
        'zipcode_98045', 'zipcode_98106', 'zipcode_98022', 'zipcode_98177',
        'zipcode_98008', 'zipcode_98002', 'zipcode_98040', 'zipcode_98029',
        'zipcode_98042', 'zipcode_98065', 'zipcode_98056', 'zipcode_98144',
        'zipcode_98188', 'zipcode_98109', 'zipcode_98010', 'zipcode_98075',
        'zipcode_98198', 'zipcode_98030', 'zipcode_98105', 'zipcode_98023',
        'zipcode_98

In [8]:
df.loc[df.sqft_living >= 7000, "sqft_living"] = 7000
df.loc[df.sqft_lot >= 500000, "sqft_lot"] = 500000
df.loc[df.bathrooms >= 6, "bathrooms"] = 6
df.loc[df.bedrooms >= 10, "bedrooms"] = 10
df.loc[df.sqft_lot15 >= 500000, "sqft_lot15"] = 500000

In [11]:
df.drop(columns=['Unnamed: 0', 'id','date'], inplace = True)

In [18]:
df['sqft_percentage'] = df['sqft_living'] / df['sqft_lot']

In [19]:
#Converting yr_built column into datetime.
df['dt_yr_built'] = pd.to_datetime(df['yr_built'].astype(str), format='%Y')

In [20]:
df['decade_built'] = ((df['dt_yr_built'].dt.year//10)*10)

In [21]:
df = pd.get_dummies(df, columns = ['zipcode'], drop_first = True)

In [22]:
df.isnull().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
                ..
zipcode_98177    0
zipcode_98178    0
zipcode_98188    0
zipcode_98198    0
zipcode_98199    0
Length: 89, dtype: int64

In [23]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,sqft_percentage,dt_yr_built,decade_built,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,0.197391,1967-01-01,1960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,0.197391,1967-01-01,1960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,47.5472,-121.998,1470,1576,0.826307,2005-01-01,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,47.7427,-122.071,1160,10565,0.079012,1976-01-01,1970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,47.4863,-122.140,2830,7916,0.348265,2005-01-01,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
final_answers = model.predict(df[other_info[0]])

In [33]:
final_answers_df = pd.DataFrame(final_answers)

In [34]:
final_answers_df.to_csv('housing_preds_mark_rubin.csv')

In [35]:
final_answers_df.describe()

,0
count,4.322000e+03
mean,5.596085e+05
std,3.241413e+05
min,-2.884622e+05
25%,3.445160e+05
50%,4.956567e+05
75%,7.031656e+05
max,2.804738e+06
